In [1]:
pip install google_play_scraper

     |████████████████████████████████| 51kB 3.2MB/s 
  Created wheel for google-play-scraper: filename=google_play_scraper-0.1.1-cp36-none-any.whl size=22260 sha256=841700abfdb33d0c59431319ca31ecc23a76ae84c134a3a32c5e9b320e166365
  Stored in directory: /root/.cache/pip/wheels/d7/1f/71/e2b30aab85297ad6dd2e3049587a6763cfb7e803a0b76d982e
Successfully built google-play-scraper


In [2]:
import os

# verifying and removing the file, if exists from google drive

if os.path.exists("app_review.csv"):
  os.remove("app_review.csv")
  print("The file is deleted.")
else:
  print("The file does not exist.")

The file does not exist.


In [ ]:
import multiprocessing
from google_play_scraper import Sort, reviews
import re
import sys


def web_scrapping(x):
  for i in range (x,x+200):
    try:
      result, continuation_token = reviews(
        'com.google.android.apps.youtube.kids',
        lang='en', # defaults to 'en'
        country='us', # defaults to 'us'
        sort=Sort.RATING, # None, defaults to Sort.MOST_RELEVANT
        count=i, # defaults to 100
        filter_score_with=5
      )

      # If we pass 'continuation_token' as an argument to the reviews function at this point,
      # it will crawl the items after "i-th" review item.
          
      result, _ = reviews(
        'com.google.android.apps.youtube.kids',
        continuation_token=continuation_token # defaults to None(load from the beginning)
      )

      # remove unicode characters from data
      txt = str(result)
      txt = remove_emoji(txt)
      txt = re.sub("'","",txt)
      txt = re.sub("\"","",txt)
      txt = re.sub("#","",txt)
      user = txt.split('userName: ')[1].split(', userImage')[0]
      rating = txt.split('score: ')[1].split(', thumbsUpCount')[0]
      upvote = txt.split('thumbsUpCount: ')[1].split(', reviewCreatedVersion')[0]
      datetime = re.search('datetime.datetime\((.+?)\),', txt).group(1)
      datetime = re.sub(", ",",",datetime)
      review = re.search('content: (.+?), score', txt).group(1)
      review = re.sub("!","",review)
      review = re.sub(",","",review)
      review = "'"+review+"'"

      print("i : ",i)
      print(user)
      print(datetime)
      print(rating)
      print(upvote)
      print(review)

      # created and opened file in append mode
      file = open('app_review.csv','a')
      file.write(user+', '+rating+', '+upvote+', '+review+', '+datetime+'\n')

    finally:
          file.close()
  return

# applying multi-processing to extract reviews 5x faster - p1, p2, p3, p4, p5
# each child process will extract 100 reviews
# we can add as many child processes as we like.

i=1
p1 = multiprocessing.Process(target=web_scrapping,args=[i])
p2 = multiprocessing.Process(target=web_scrapping,args=[i+100])
p3 = multiprocessing.Process(target=web_scrapping,args=[i+200])
p4 = multiprocessing.Process(target=web_scrapping,args=[i+300])
p5 = multiprocessing.Process(target=web_scrapping,args=[i+400])


# starting the child processes

p1.start()
p2.start()
p3.start()
p4.start()
p5.start()


# joining the child processes to parent process

p1.join()
p2.join()
p3.join()
p4.join()
p5.join()


In [ ]:
# downloading the .csv file into local drive

from google.colab import files
files.download('app_review.csv')